<a href="https://colab.research.google.com/github/JeanCarloTS/Machine-Learning-Projects/blob/main/Est%C3%A1gio/ML_Dados_Twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Bibliotecas**

In [1]:
import pandas as pd                   # manipulação de base de dados
import numpy as np                    # manipulação numérica
import seaborn as sns                 # manipulaçõa gráfica
import matplotlib.pyplot as plt       # manipulação gráfica

# **2. Importação dos dados**

## **2.1 Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Estágio - IMT & NPL Brasil/Notebooks/DataFrame_final_Twitter.csv', error_bad_lines=False)

## **2.2 Local**

In [2]:
df = pd.read_csv(r'C:\Users\jcts1\Desktop\JEAN\GitHub\Machine-Learning-Projects\Estágio\Twitter\Arquivos_e_dados_finais\DataFrame_final_Twitter.csv', error_bad_lines=False)

In [3]:
df

,text,likes_faixa,likes
0,Tremores de terra sacudiram Goma na República ...,-1,7
1,O novo presidente do Equador Guillermo Lasso f...,-1,13
2,Novos dados sobre pesquisadores de Wuhan aumen...,0,39
3,RT A Ericsson disse que pode obter uma fatia ...,-1,0
4,Aeroportos devem ser foco para conter a varian...,0,42
...,...,...,...
16235,Este conteúdo faz parte da NexoEDUnoEnem noss...,-1,26
16236,AcessoLivre Ao longo do mês de dezembro o Ne...,-1,19
16237,Este conteúdo faz parte da NexoEDUnoEnem noss...,-1,23
16238,opinião O termo empreendedorismo nada mais é d...,0,62


In [5]:
df_copy = df.copy()
df_copy

,text,likes_faixa,likes
0,Tremores de terra sacudiram Goma na República ...,-1,7
1,O novo presidente do Equador Guillermo Lasso f...,-1,13
2,Novos dados sobre pesquisadores de Wuhan aumen...,0,39
3,RT A Ericsson disse que pode obter uma fatia ...,-1,0
4,Aeroportos devem ser foco para conter a varian...,0,42
...,...,...,...
16235,Este conteúdo faz parte da NexoEDUnoEnem noss...,-1,26
16236,AcessoLivre Ao longo do mês de dezembro o Ne...,-1,19
16237,Este conteúdo faz parte da NexoEDUnoEnem noss...,-1,23
16238,opinião O termo empreendedorismo nada mais é d...,0,62


In [6]:
df_copy.columns                            # Informação de quais são as colunas do Data Frame, e também, como estão nomeadas

Index(['text', 'likes_faixa', 'likes'], dtype='object')

In [7]:
df_copy['text'][16238]

'opinião O termo empreendedorismo nada mais é do que a expressão invertida da eliminação de qualquer rede de prot… '

# **3. Separando os dados para treino e teste**

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_copy['text'],df_copy['likes_faixa'], test_size = 0.25)

# **4. Definindo o vocabulário com TfidfVectorizer**

In [8]:
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')
language = "portuguese"
stopwords_list  = stopwords.words(language)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jcts1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [272]:
### TfidfVectorizer é melhor para datasets maiores(muito grandes), além disso possui mais features para stop_words, lowercase, frequency e etc.
from sklearn.feature_extraction.text import TfidfVectorizer
max_freq        =   0.3

corpus          =   np.array(X_train)    
vectorizer_tf   =   TfidfVectorizer(min_df = 2, max_df = max_freq, stop_words = stopwords_list)
tfidfvec        =   vectorizer_tf.fit_transform(corpus.astype('U'))   # Contabiliza a frequencia de uma palavra dentro de uma mensagem em relação a quantas aparições no total ela tem
array_vec       =   tfidfvec.toarray()                    # Matriz de arrays com as frequências de uma palavra aprendida em uma determinada mensagem
words_lrnd2     =   vectorizer_tf.get_feature_names()     # Palavras aprendidas com os dados 

In [273]:
print("\n",len(words_lrnd2))


 4200


# **5. Aplicando algoritimos de Machine Learning nos dados**

## **5.1 Multinomial Naive Bayes**

In [283]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics                                 # Biblioteca para medir a acurácia do modelo

### Classificador
clf_nb   =  MultinomialNB(alpha = 0.1)                         # Alpha = 1 (Suavização de Laplace) 

### Treinamento
clf_nb.fit(tfidfvec, y_train)

MultinomialNB(alpha=0.1)

In [291]:
vector_pred1     =   clf_nb.predict(tfidfvec)
acc_score = metrics.accuracy_score(y_train,vector_pred1)     # Teste de acurácia
print("A porcentagem de acerto na faixa de likes usando os parâmetros de teste foi {:.2f}%".format(100*acc_score))

A porcentagem de acerto na faixa de likes usando os parâmetros de teste foi 79.28%


In [285]:
# Transforma o dado de teste para o mesmo formato do dado de treino
corpus_t        =   np.array(X_test) 
vectors_test    =   vectorizer_tf.transform(corpus_t.astype('U'))
vector_pred     =   clf_nb.predict(vectors_test)                           # Predição do modelo

In [286]:
acc_score = metrics.accuracy_score(y_test,vector_pred)     # Teste de acurácia

In [287]:
print("A porcentagem de acerto na faixa de likes usando os parâmetros de teste foi {:.2f}%".format(100*acc_score))

A porcentagem de acerto na faixa de likes usando os parâmetros de teste foi 49.19%


In [288]:
### Verificando o desempenho
vectors_test2   =   vectorizer_tf.fit_transform(corpus_t.astype('U'))

clf_nb2   =  MultinomialNB(alpha = 0.1) 
clf_nb2.fit(vectors_test2, y_test)

vector_pred2  = clf_nb2.predict(vectors_test2)

In [289]:
acc_score = metrics.accuracy_score(y_test,vector_pred2) 
acc_score

0.8810344827586207

# **5.2 Gaussian Mixture Model**

In [306]:
from sklearn.mixture import GaussianMixture

clf_gm      =   GaussianMixture()

clf_gm.fit(array_vec, y_train)

gm_pred     =   clf_gm.predict(array_vec)

In [297]:
acc_score = metrics.accuracy_score(y_train,gm_pred)     # Teste de acurácia
print("A porcentagem de acerto na faixa de likes usando os parâmetros de teste foi {:.2f}%".format(100*acc_score))

A porcentagem de acerto na faixa de likes usando os parâmetros de teste foi 26.40%


# **5.3 PCA**

In [308]:
from sklearn.decomposition import PCA

# Verificando a importancia de cada dimensão
pca = PCA()
pca.fit(array_vec)

PCA()

In [312]:
features = range(pca.n_components_)

from plotly import graph_objects as go

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x = list(features),
        y = 1 - pca.explained_variance_ratio_
    )
)

fig.show()

# **5.4 T-SNE**

In [317]:
from sklearn.manifold import TSNE

tsne = TSNE(learning_rate=50)
tsne_features   =   tsne.fit_transform(array_vec)



KeyboardInterrupt: 

# **6. Avaliando a acurácia do(s) modelo(s)**